In [1]:
class Cohort:
    """
    Defines a cohort of individuals flagged for AAA medial screening.
    """
    def __init__(self, scans, positives, rupture_rate=0.25, death_b4_treatment=0.50, emerg_surgery_success_rate=0.771,
                 total_cohort_size=900, scan_cost=400, proph_surgery_cost=31412, proph_surgery_payment=33229,
                 emerg_surgery_cost=38800, emerg_surgery_payment=39187, use_default_positivity_rate=True):
        """
        Initializes the Cohort with given parameters.

        Default probabilities and success rates from literature. Prices are based on median values from EHR.
        """

        # Assertions for input validation
        assert positives <= scans <= total_cohort_size, "Invalid input: positives <= scans <= total_cohort_size must be true"
        assert 0 <= rupture_rate <= 1.00, "Invalid rupture_rate: must be between 0 and 1"
        assert 0 <= death_b4_treatment <= 1.00, "Invalid death_b4_treatment rate: must be between 0 and 1"
        assert 0 <= emerg_surgery_success_rate <= 1.00, "Invalid emerg_surgery_success_rate: must be between 0 and 1"

        self.positives = positives
        self.scans = scans

        # from research
        self.rupture_rate = rupture_rate
        self.death_b4_treatment = death_b4_treatment
        self.emerg_surgery_success_rate = emerg_surgery_success_rate

        # program definition of size and program assumption on positivity
        self.total_cohort_size = total_cohort_size
        self.use_default_positivity_rate = use_default_positivity_rate

        # representative cost assumptions
        self.scan_cost = scan_cost
        self.proph_surgery_cost = proph_surgery_cost
        self.emerg_surgery_cost = emerg_surgery_cost

        # representative payments assumptions
        self.proph_surgery_payment = proph_surgery_payment
        self.emerg_surgery_payment = emerg_surgery_payment

    def scanning_costs(self):
        """
        Calculate and return the total cost of scanning.

        :return: Total cost of scanning.
        """
        total = self.scans * self.scan_cost
        return total

    def scanned_positivity_rate(self):
        """
        Calculate and return the positivity rate of those scanned.

        :return: Positivity rate.
        """
        rate = self.positives / self.scans
        return rate

    def _get_positivity_rate(self):
        """
        Get the positivity rate, either the default or calculated.

        :return: Positivity rate.
        """
        if self.use_default_positivity_rate:
            return 27/416 #reflects positivity rate of the program so far
        else:
            return self.scanned_positivity_rate()

    def proph_surgeries(self):
        """
        Calculate and return the number of prophylactic surgeries needed based on positivity.

        :return: Number of prophylactic surgeries.
        """
        surgeries = self.positives * self.rupture_rate
        return surgeries

    def proph_surgery_costs(self):
        """
        Calculate and return the total cost for prophylactic surgeries.

        :return: Total cost for prophylactic surgeries.
        """
        total = self.proph_surgeries() * self.proph_surgery_cost
        return total

    def proph_surgery_payments(self):
        """
        Calculate and return the total cost for prophylactic surgeries.

        :return: Total cost for prophylactic surgeries.
        """
        total = self.proph_surgeries() * self.proph_surgery_payment
        return total

    def untreated_deaths(self):
        """
        Calculate and return the number of untreated deaths in the cohort.

        :return: Number of untreated deaths.
        """
        positivity_rate = self._get_positivity_rate()

        deaths = (self.total_cohort_size - self.scans) * self.rupture_rate * self.death_b4_treatment * positivity_rate
        return deaths

    def emerg_surgeries(self):
        """
        Calculate and return the number of emergency surgeries needed.

        :return: Number of emergency surgeries.
        """
        positivity_rate = self._get_positivity_rate()

        surgeries = (self.total_cohort_size - self.scans) * self.rupture_rate * (1 - self.death_b4_treatment) * positivity_rate
        return surgeries

    def emerg_surgery_costs(self):
        """
        Calculate and return the total cost for emergency surgeries.

        :return: Total cost for emergency surgeries.
        """
        total = self.emerg_surgeries() * self.emerg_surgery_cost
        return total

    def emerg_surgery_payments(self):
        """
        Calculate and return the total cost for emergency surgeries.

        :return: Total cost for emergency surgeries.
        """
        total = self.emerg_surgeries() * self.emerg_surgery_payment
        return total

    def emerg_surgery_deaths(self):
        """
        Calculate and return the number of deaths resulting from emergency surgeries.

        :return: Number of deaths from emergency surgeries.
        """
        deaths = self.emerg_surgeries() * (1 - self.emerg_surgery_success_rate)
        return deaths

    def total_cost(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.scanning_costs() + self.proph_surgery_costs() + self.emerg_surgery_costs()
        return total

    def total_payments(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.scanning_costs() + self.proph_surgery_payments() + self.emerg_surgery_payments()
        return total

    def total_income(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.total_payments() - self.total_cost()
        return total

    def total_deaths(self):
        """
        Calculate and return the total number of deaths in the cohort.

        :return: Total number of deaths.
        """
        total = self.untreated_deaths() + self.emerg_surgery_deaths()
        return total

    def full_results(self,printout=True):
        """
        Aggregate and format results from various methods to display a comprehensive overview of the cohort's outcomes.

        :return: None, but prints the results.
        """
        methods_to_call = ['emerg_surgeries', 'emerg_surgery_costs', 'emerg_surgery_payments', 'emerg_surgery_deaths',
                           'scanned_positivity_rate', 'proph_surgeries', 'proph_surgery_costs', 'proph_surgery_payments',
                           'scanning_costs', 'total_cost', 'total_payments', 'total_income','total_deaths',
                           'untreated_deaths']

        results = {}

        for method_name in methods_to_call:
            method = getattr(self, method_name)
            results[method_name] = method()

        # Formatting the values
        results['emerg_surgeries'] = round(results['emerg_surgeries'], 1)
        results['emerg_surgery_deaths'] = round(results['emerg_surgery_deaths'], 1)
        results['proph_surgeries'] = round(results['proph_surgeries'], 1)
        results['total_deaths'] = round(results['total_deaths'], 1)
        results['untreated_deaths'] = round(results['untreated_deaths'], 1)

        results['emerg_surgery_costs'] = "${:,.0f}".format(results['emerg_surgery_costs'])
        results['proph_surgery_costs'] = "${:,.0f}".format(results['proph_surgery_costs'])
        results['emerg_surgery_payments'] = "${:,.0f}".format(results['emerg_surgery_payments'])
        results['proph_surgery_payments'] = "${:,.0f}".format(results['proph_surgery_payments'])

        results['scanning_costs'] = "${:,.0f}".format(results['scanning_costs'])
        results['total_cost'] = "${:,.0f}".format(results['total_cost'])
        results['total_payments'] = "${:,.0f}".format(results['total_payments'])
        results['total_income'] = "${:,.0f}".format(results['total_income'])

        results['scanned_positivity_rate'] = "{:.1%}".format(results['scanned_positivity_rate'])

        if printout==False:
            return results
        else:
            for key, value in results.items():
                print(f"{key}: {value}")
                print("-------------------------------")

In [3]:
test = Cohort(416,60)
test.full_results()

emerg_surgeries: 3.9
-------------------------------
emerg_surgery_costs: $152,355
-------------------------------
emerg_surgery_payments: $153,875
-------------------------------
emerg_surgery_deaths: 0.9
-------------------------------
scanned_positivity_rate: 14.4%
-------------------------------
proph_surgeries: 15.0
-------------------------------
proph_surgery_costs: $471,180
-------------------------------
proph_surgery_payments: $498,435
-------------------------------
scanning_costs: $166,400
-------------------------------
total_cost: $789,935
-------------------------------
total_payments: $818,710
-------------------------------
total_income: $28,775
-------------------------------
total_deaths: 4.8
-------------------------------
untreated_deaths: 3.9
-------------------------------
